In [1]:
import pandas as pd
import json
from pathlib import Path
from collections import Counter

In [2]:
problems_path = Path("data/aeri/problems_extract.csv")

In [3]:
problems_df = pd.read_csv(problems_path)

In [15]:
problems_df.head()

,summary,numberOfReporters,numberOfIncidents,v1status,kind,createdOn,modifiedOn,savedOn,osgiArch,osgiOs,osgiOsVersion,osgiWs,eclipseBuildId,eclipseProduct,javaRuntimeVersion
0,SocketException below JDKHttpConnection.getRes...,2,3.0,CONFIRMED,NORMAL,2014-09-12T08:47:00.888Z,2016-06-15T06:39:43.662Z,2016-09-19T08:39:33.025Z,x86_64,Windows7,6.1.0,win32,3.6.1.201408250818-RELEASE-e44,org.springsource.sts.ide,1.8.0_05-b13
1,BundleException below BundleModel.load (thrown...,74,0.0,INVALID,NORMAL,2016-03-15T15:43:15.704Z,2016-12-07T23:22:56.032Z,2016-12-11T07:31:47.499Z,x86_64,MacOSX,10.9.4,cocoa,4.4.0.I20140606-1215,org.eclipse.epp.package.standard.product,1.8.0-b132
2,NullPointerException in TypeReference.internal...,91,11.0,FIXED,NORMAL,2016-03-13T10:51:10.204Z,2016-12-12T09:47:21.067Z,2016-12-12T09:47:21.332Z,x86_64,MacOSX,10.9.4,cocoa,4.4.0.I20140606-1215,org.eclipse.epp.package.standard.product,1.8.0-b132
3,3rd party caused VerifyError below XtendActiva...,0,0.0,INVALID,THIRD,2014-09-12T17:41:11.216Z,2014-09-12T17:41:11.216Z,2016-09-19T08:39:33.461Z,x86_64,MacOSX,10.9.4,cocoa,4.4.0.I20140606-1215,org.eclipse.epp.package.standard.product,1.8.0_05-b13
4,NoStackTrace in MenuHelper$1.evaluate,895,1351.0,INVALID,NORMAL,2015-12-22T17:43:07.104Z,2016-12-13T12:02:20.232Z,2016-12-13T12:02:20.358Z,x86_64,MacOSX,10.9.4,cocoa,4.4.0.I20140606-1215,org.eclipse.epp.package.standard.product,1.8.0-b132


In [26]:
problems_df.summary.value_counts()

summary
FreezeWarning in OS.objc_msgSend                                                                                         32
3rd party caused NullPointerException below HandlerProxy.execute                                                         32
FreezeWarning below EventTable.sendEvent (thrown in ZipFile.read)                                                        30
FreezeWarning below EventTable.sendEvent (thrown in HIDDEN)                                                              28
3rd party caused OutOfMemoryError below Worker.run                                                                       24
                                                                                                                         ..
StackOverflowError below DeclaratorTypeArgumentCollector.doVisitParameterizedTypeReference (thrown in HashMap.putVal)     1
StackOverflowError in LightweightTypeReferenceFactory.doVisitParameterizedTypeReference                                   1


In [27]:
if 'osgiOs' in problems_df.columns:
        print("\n💻 OS Distribution:")
        print("-" * 40)
        os_counts = problems_df['osgiOs'].value_counts().head(5)
        for os_name, count in os_counts.items():
            print(f"   {os_name}: {count:,} ({100*count/len(problems_df):.1f}%)")


💻 OS Distribution:
----------------------------------------
   Windows7: 18,060 (35.5%)
   Linux: 11,360 (22.3%)
   Windows10: 7,420 (14.6%)
   Windows8: 6,788 (13.3%)
   MacOSX: 6,562 (12.9%)


In [28]:
incidents_path = Path("data/aeri/incidents_extract.csv")
incidents_df = pd.read_csv(incidents_path)

In [29]:
incidents_df.shape

(2084045, 20)

In [30]:
problems_df.shape

(50872, 15)

In [31]:
incidents_df.head()

,kind,severity,pluginId,pluginVersion,code,statusFingerprint,message,statusSeverity,fingerprint,fingerprint2,timestamp,savedOn,osgiArch,osgiOs,osgiOsVersion,osgiWs,eclipseBuildId,eclipseProduct,javaRuntimeVersion,commentQuality
0,NORMAL,UNKNOWN,org.eclipse.jdt.launching,3.8.100.v20160505-0636,0,98631af2ddb2d197ebdca532f19d082b,Failed to retrieve default libraries for C:\Pr...,4,cd03d068798d141412b1d1605892fbec,12166d864efb7adcccc187034deb7dbf,2016-11-08T10:22:59.204Z,2016-11-08T10:23:01.914Z,x86_64,Windows7,6.1.0,win32,4.6.1.M20160907-1200,org.eclipse.epp.package.jee.product,1.8.0_112-b15,UNKNOWN
1,NORMAL,UNKNOWN,org.eclipse.core.jobs,3.8.0.v20160509-0411,2,7d8ffe0da7847c69362bfe02a725e802,An internal error occurred during: 'Requesting...,4,a8c1254b60314cc36f8998c6dfef98f6,ad49c3cfd0b92eaf7ae875a816d1acf4,2016-11-08T10:23:25.019Z,2016-11-08T10:23:25.058Z,x86_64,Windows7,6.1.0,win32,4.6.1.M20160907-1200,org.eclipse.epp.package.jee.product,1.8.0_112-b15,UNKNOWN
2,NORMAL,UNKNOWN,org.eclipse.ui.workbench,3.108.1.v20160819-2118,2,448fa1a583c4f5b1e9e597d16526e5b7,Problems occurred when invoking code from plug...,4,0d419e9729410186a626ef406d1faac4,61958756ccdb85d54e49ec9b9eee30bf,2016-11-08T10:23:27.229Z,2016-11-08T10:23:27.233Z,x86_64,Windows7,6.1.0,win32,4.6.1.M20160907-1200,org.eclipse.epp.package.jee.product,1.8.0_112-b15,UNKNOWN
3,NORMAL,UNKNOWN,org.eclipse.ui.workbench,3.108.1.v20160819-2118,2,ae47c6aaac5129e6716cbfb78da6a0b0,Problems occurred when invoking code from plug...,4,0602448fb3dc6b6f0011bc947537e936,0c24855337157c0e979cc3f7598c0d27,2016-11-08T10:23:27.485Z,2016-11-08T10:23:27.520Z,x86_64,Windows7,6.1.0,win32,4.6.1.M20160907-1200,org.eclipse.epp.package.jee.product,1.8.0_112-b15,UNKNOWN
4,NORMAL,UNKNOWN,org.eclipse.core.jobs,3.8.0.v20160509-0411,2,f2810736828fc27205678a70c4189508,'hws-web のステータスを更新中...' 中に内部エラーが発生しました。,4,0d96e5686e8c180ee0dda71cdbef0942,350749f2a4dbe1d69def81648cad96f7,2016-11-08T10:23:34.320Z,2016-11-08T10:23:34.363Z,x86_64,Windows8,6.3.0,win32,4.6.1.M20160907-1200,org.eclipse.epp.package.jee.product,1.8.0_31-b13,UNKNOWN


In [32]:
lite_path = Path("data/swebench/swebench_lite.json")

In [44]:
with open(lite_path) as f:
    swebench_data = json.load(f)

In [45]:
sample = swebench_data[0]

In [46]:
for key in sample.keys():
    val = sample[key]
    val_type = type(val).__name__
    val_preview = str(val)[:60] + "..." if len(str(val)) > 60 else str(val)
    print(f" - {key} ({val_type}): {val_preview}")


 - repo (str): astropy/astropy
 - instance_id (str): astropy__astropy-12907
 - base_commit (str): d16bfe05a744909de4b27f5875fe0d4ed41ce607
 - patch (str): diff --git a/astropy/modeling/separable.py b/astropy/modelin...
 - test_patch (str): diff --git a/astropy/modeling/tests/test_separable.py b/astr...
 - problem_statement (str): Modeling's `separability_matrix` does not compute separabili...
 - hints_text (str): 
 - created_at (str): 2022-03-03T15:14:54Z
 - version (str): 4.3
 - FAIL_TO_PASS (str): ["astropy/modeling/tests/test_separable.py::test_separable[c...
 - PASS_TO_PASS (str): ["astropy/modeling/tests/test_separable.py::test_coord_matri...
 - environment_setup_commit (str): 298ccb478e6bf092953bca67a3d29dc6c35f6752


In [55]:
repos = Counter(item['repo'] for item in swebench_data)

for repo, count in repos.most_common():
    print(f"   {repo}: {count} issues")

   django/django: 114 issues
   sympy/sympy: 77 issues
   matplotlib/matplotlib: 23 issues
   scikit-learn/scikit-learn: 23 issues
   pytest-dev/pytest: 17 issues
   sphinx-doc/sphinx: 16 issues
   astropy/astropy: 6 issues
   psf/requests: 6 issues
   pylint-dev/pylint: 6 issues
   pydata/xarray: 5 issues
   mwaskom/seaborn: 4 issues
   pallets/flask: 3 issues


In [ ]:
for item in swebench_data:
    if 'hints_text' in item and item['hints_text'] != '':
        print(f"Index: {swebench_data.index(item)}")
        print(item['hints_text'])
        break

Index: 2
Welcome to Astropy 👋 and thank you for your first issue!

A project member will respond to you as soon as possible; in the meantime, please double-check the [guidelines for submitting issues](https://github.com/astropy/astropy/blob/main/CONTRIBUTING.md#reporting-issues) and make sure you've provided the requested details.

GitHub issues in the Astropy repository are used to track bug reports and feature requests; If your issue poses a question about how to use Astropy, please instead raise your question in the [Astropy Discourse user forum](https://community.openastronomy.org/c/astropy/8) and close this issue.

If you feel that this issue has not been responded to in a timely manner, please send a message directly to the [development mailing list](http://groups.google.com/group/astropy-dev).  If the issue is urgent or sensitive in nature (e.g., a security vulnerability) please send an e-mail directly to the private e-mail feedback@astropy.org.
Huh, so we do have this format...

In [71]:
example = swebench_data[2]

problem = example['problem_statement']

In [75]:
hints = example.get('hints_text', '')

In [76]:
hints

"Welcome to Astropy 👋 and thank you for your first issue!\n\nA project member will respond to you as soon as possible; in the meantime, please double-check the [guidelines for submitting issues](https://github.com/astropy/astropy/blob/main/CONTRIBUTING.md#reporting-issues) and make sure you've provided the requested details.\n\nGitHub issues in the Astropy repository are used to track bug reports and feature requests; If your issue poses a question about how to use Astropy, please instead raise your question in the [Astropy Discourse user forum](https://community.openastronomy.org/c/astropy/8) and close this issue.\n\nIf you feel that this issue has not been responded to in a timely manner, please send a message directly to the [development mailing list](http://groups.google.com/group/astropy-dev).  If the issue is urgent or sensitive in nature (e.g., a security vulnerability) please send an e-mail directly to the private e-mail feedback@astropy.org.\nHuh, so we do have this format... 

In [77]:
example

{'repo': 'astropy/astropy',
 'instance_id': 'astropy__astropy-14365',
 'base_commit': '7269fa3e33e8d02485a647da91a5a2a60a06af61',
 'patch': 'diff --git a/astropy/io/ascii/qdp.py b/astropy/io/ascii/qdp.py\n--- a/astropy/io/ascii/qdp.py\n+++ b/astropy/io/ascii/qdp.py\n@@ -68,7 +68,7 @@ def _line_type(line, delimiter=None):\n     _new_re = rf"NO({sep}NO)+"\n     _data_re = rf"({_decimal_re}|NO|[-+]?nan)({sep}({_decimal_re}|NO|[-+]?nan))*)"\n     _type_re = rf"^\\s*((?P<command>{_command_re})|(?P<new>{_new_re})|(?P<data>{_data_re})?\\s*(\\!(?P<comment>.*))?\\s*$"\n-    _line_type_re = re.compile(_type_re)\n+    _line_type_re = re.compile(_type_re, re.IGNORECASE)\n     line = line.strip()\n     if not line:\n         return "comment"\n@@ -306,7 +306,7 @@ def _get_tables_from_qdp_file(qdp_file, input_colnames=None, delimiter=None):\n \n             values = []\n             for v in line.split(delimiter):\n-                if v == "NO":\n+                if v.upper() == "NO":\n              